In [45]:
import pickle
import numpy as np
import pandas as pd
import itertools
from mastml.mastml import Mastml
from mastml.feature_generators import ElementalFeatureGenerator, ElementalFractionGenerator
# from mastml.feature_generators import ElementalFeatureGenerator


In [2]:

pfile_X = "Codes/bandgap_pbe_X.pickle"
pfile_Y = "Codes/bandgap_pbe_Y.pickle"

with open(pfile_X, 'rb') as f:
    data = pickle.load(f)

In [6]:
# for k,v in data.items():
    # print(k)

# n = 74992
# acc = 0
# for id in data['icsd_id']:
#     if not id:
#         acc += 1

# print(acc / n)
# print(list(numpy.unique(data['reference'])))
# print(list(numpy.unique(data['comments'])))
# print(list(numpy.unique(data['bandgap type'])))
# print(list(numpy.unique(data['comp method'])))

x_clean = data[['composition', 'structure', 'space group']]
# print("x_clean", x_clean)

composition = x_clean['composition']
structure = x_clean['structure']

m = float("inf")

# foo = structure[m]
# compositions = [composition[i] for i in range(m)]
compositions = [comp for i, comp in enumerate(composition) if i < m and "O" in comp]

# sites = foo['sites']
# print("compositions", compositions)
print("num compositions", len(compositions))

# l = []
# for s in sites:
#     l.append(s['species'])

# element_names = ['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K',
#                  'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb',
#                  'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs',
#                  'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta',
#                  'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa',
#                  'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt',
#                  'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']

# print(bar)
# decomposeComposition(bar)
# print(bar)
# EnsembleModelFeatureSelector()
# generate_elementfraction_features(bar)

num compositions 34662


In [23]:
# df = pd.DataFrame([compositions])
df = pd.DataFrame(compositions)
# print("df", df)
# ! USE ElementFraction instead ElementalFractionGenerator
# e = ElementalFeatureGenerator(df)
e = ElementalFractionGenerator(df)
e.fit()
x, y = e.transform()
print("vector shape", x.shape, "y", y)


vector shape (34662, 118) y None


In [59]:
# list(x.items())
# print(*x.items(), sep="\n")
# print(x.iloc[0])

## verify rows sum to 1
# total = x.sum(axis=1, numeric_only= True)
# print("total", total)


thresholds = [i / 10 for i in range(10)]
for th in thresholds:
    # print("threshold", th)
    O_group = x[x['O'] > th]
    not_O_group = x[x['O'] <= th]
    # print("o group shape for theshold", th, O_group.shape, O_group.shape[0]/x.shape[0])
    print("theshold", th, "; percent in O_group", O_group.shape[0]/x.shape[0])
    # print("not o group shape for theshold", th, not_O_group.shape, not_O_group.shape[0]/x.shape[0])

    # th = 0.5
    O_group = x[x['O'] > th]
    not_O_group = x[x['O'] <= th]

    o_cent = O_group.mean(axis=0)
    not_o_cent = not_O_group.mean(axis=0)
    x_cent = x.mean(axis=0)

    # groups = (o_cent, not_o_cent, x_cent)
    # group_names = ("o_cent", "not_o_cent", "x_cent")
    groups = {"o_cent": o_cent, "not_o_cent": not_o_cent, "x_cent": x_cent}

    pairs = list(itertools.combinations(list(groups.items()), 2))
    # print(*pairs, sep="\n")

    for ((g1_name, g1), (g2_name, g2)) in pairs:
        dist = np.sqrt(np.sum([(a-b)*(a-b) for a, b in zip(g1, g2)]))
        print(f"dist {g1_name} {g2_name}", dist)
    print()

# print("o_cent", o_cent)
# print("not_o_cent", not_o_cent)
# print("x_cent", x_cent)


theshold 0.0 percent in O_group 0.9833246783220818
dist o_cent not_o_cent 0.6363567362711182
dist o_cent x_cent 0.010611453279230994
dist not_o_cent x_cent 0.6257452829918871

theshold 0.1 percent in O_group 0.9656107552939819
dist o_cent not_o_cent 0.5529246548514809
dist o_cent x_cent 0.019014661259676643
dist not_o_cent x_cent 0.5339099935918042

theshold 0.2 percent in O_group 0.9242109514742369
dist o_cent not_o_cent 0.5100590649617123
dist o_cent x_cent 0.03865689122539
dist not_o_cent x_cent 0.4714021737363223

theshold 0.3 percent in O_group 0.8901390571807744
dist o_cent not_o_cent 0.47176091118859814
dist o_cent x_cent 0.051828098488441915
dist not_o_cent x_cent 0.41993281270015625

theshold 0.4 percent in O_group 0.824274421556748
dist o_cent not_o_cent 0.4005319956672001
dist o_cent x_cent 0.070383716623667
dist not_o_cent x_cent 0.3301482790435331

theshold 0.5 percent in O_group 0.672840574692747
dist o_cent not_o_cent 0.2969931546409783
dist o_cent x_cent 0.0971641097925

In [ ]:
## Compute distances